# Embedding Service

In this notebook will be tested the functionality of the embedding service.

This service must do two main steps:

- chunk a raw text
- embed the chunked text into vectors

To do so, both services requires of an [*embedding model*](https://huggingface.co/blog/getting-started-with-embeddings):

- chunking: Uses the embedding model to generate tokens, this is due to tokenization can has multiple approaches.
- embedding: Uses the embedding model to create the vectors that will represent the data of each chunk generated.

In [24]:
from langchain_text_splitters import TokenTextSplitter
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np
import uuid
from datetime import datetime
import requests

import sys

sys.path.append("..")

from rag_llm_energy_expert.services.embeddings.config import EmbeddingsConfig

In [25]:
embeddings_config = EmbeddingsConfig()

The following text will be used:

In [26]:
text = """
Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-\ntividades de exploración y extracción de petróleo y gas natural.\n4.\t\nPermitir que la Nación ejerza, de manera exclusiva, la planeación y control del Sistema \nEléctrico Nacional, en beneficio de un sistema competitivo que permita reducir los precios \nde la energía eléctrica.\n5.\t\nAtraer mayor inversión al sector energético mexicano para impulsar el desarrollo del país.\n6.\t\nContar con un mayor abasto de energéticos a mejores precios.\n7. \t Garantizar estándares internacionales de eficiencia, calidad y confiabilidad de suministro \nenergético, así como transparencia y rendición de cuentas en las distintas actividades de la \nindustria energética.\n8.\t\nCombatir de manera efectiva la corrupción en el sector energético.\n9.\t\nFortalecer la administración de los ingresos petroleros e impulsar el ahorro de largo plazo en \nbeneficio de las futuras generaciones.\n10.\t Impulsar el desarrollo, con responsabilidad social y ambiental.\nEstos objetivos se verán traducidos en beneficios concretos para los mexicanos:\n
"""

print(text)


Resumen Ejecutivo


3
I. Introducción
La Reforma Energética es un paso decidido rumbo a la modernización del sector energético de 
nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-
to de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a 
nivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-
das por los partidos políticos representados en el Congreso.
La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:
1.	
Mantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-
suelo.
2.	
Modernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-
deral de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% 
mexicanas.
3.	
Reducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-
tividades de exploración y extracción de

## Chunking

**Chunking depends of how many tokens an embedding model supports**

There's a [HuggingFace dashboard](https://huggingface.co/spaces/mteb/leaderboard) that compares the performance of different embedding models. Some metrics to focus on are:

- Number of Parameters: 

    A higher value means the model requires more CPU/GPU memory to run

- Embedding Dimension:

    The dimension of the vectors produced

- Max tokens:

    How many tokens the model can process, the higher the better.


For this time, we'll be using the [*nomic-ai/nomic-embed-text-v2-moe*](https://huggingface.co/nomic-ai/nomic-embed-text-v2-moe) model:

- Number of Parameters: 475M
- Embedding Dimension: 768
- Max tokens: 512



[*Langchain*](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html#textsplitter) contains many *text splitters*. Most of them uses string characters to split the data. Nevertheless, an *embedding model* uses *tokens* to delimit the amount of text to be embedded into a vector. To fix this, we can use a Langchain [*TokenTextSplitter*](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TokenTextSplitter.html) class.


The *TokenTextSplitter* object splits the text into tokens using a model tokenizer. We can use a [*HuggingFace Tokenizer*](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) as an imput parameter for the TokenTextSplitter. A *tokenizer* is in charge of preparing the inputs for a model, this is, convert the text into tokens, then the TokenTextSplitter split the tokens into the chunk_size (max tokens that an embedding model can handle), and then the tokenizer decode the tokens to convert them again into text. 

The [*HuggingFace Tokenizer*](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) library contains tokenizers for all the embedding models.

In [27]:
model = SentenceTransformer(embeddings_config.EMBEDDING_MODEL, trust_remote_code=True)

In [28]:
# Initialize an AutoTokenizer instance, this will tokenize the data based on 
# the embedding model used
tokenizer = AutoTokenizer.from_pretrained(embeddings_config.EMBEDDING_MODEL)

# Create a TokenTextSplitter instance, this will use the Tokenizer to
# tokenize the text, split it based on the max tokens supported by the
# embedding model, and then convert the tokens into text again, but now splitted.
splitter = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=model.max_seq_length,
    chunk_overlap = embeddings_config.CHUNK_OVERLAP,
)

In [29]:
# Returns a list of strings, each entry of the list is a chunk
chunks = splitter.split_text(text)

In [30]:
chunks

['\nResumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petr',
 'ecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambie

In [31]:
chunks_sizes = [len(chunk) for chunk in chunks]

print(f"Number of chunks: {len(chunks)} \n"
      f"mean chunk size (in characters): {np.mean(chunks_sizes):.4f}\n"
      f"max chunk size (in characters): {max(chunks_sizes)}"
      )

Number of chunks: 3 
mean chunk size (in characters): 676.3333
max chunk size (in characters): 719


## Embedding

Now that the data is splitted based on the chunk size of each embedding model. We must notice that a vector DB is the place where we will store the chunks to create the semantic search, to do so, each chunk must be embedded into a vector using the embedding model that fits the chunking size.

There are 3 key elements that define a vector in a vector DB:

- ID
- Dimensions
- Payload

The payload contains all the metadata and the text that was embedded in the vector DB. This is because once the text has been encoded, you cannot retrieve the original data from the vector. So it is necessary to store the text as metadata.

In the next sections, we'll be adding metadata to the chunks created.

In [32]:
# Adding info such as title, and the date of chunking
metadata = {
    "upload_date": datetime.now().strftime(r"%Y-%m-%d"),
    "title": "Title of the text",
    "storage_path": "path/where/the/text/was/stored.pdf",
    }

Then, from the list of strings obtained (chunks), it will be created a more structured chunk, to comply with the necessary of each chunk to be indexed in any vector database

In [33]:
# Embedding the chunk text using batch embedding
chunks_embedded = model.encode(chunks)

# Create a list of dictionaries, which each dictionary is a chunk with all the necessary to be
# indexed into a vector DB
final_chunks = [
    {
        "id": str(uuid.uuid4()),
        "vector": chunks_embedded[i],
        "payload": {
            "text": chunk_text,
            "metadata": metadata,
        },
    }
    for i, chunk_text in enumerate(chunks)
]

In [34]:
final_chunks[0]

{'id': 'c8390ec1-1c7a-4893-8763-8ad1200cde7b',
 'vector': array([ 2.59235669e-02,  2.15934832e-02, -5.43275476e-02, -1.02517745e-02,
         8.65479888e-05,  1.96502870e-03, -5.38319582e-03, -2.79700123e-02,
        -5.68942577e-02,  3.46788019e-02,  2.07133107e-02,  2.04562545e-02,
        -4.09352779e-02, -4.51724641e-02,  8.72723088e-02, -5.99049171e-03,
        -2.09121685e-02,  1.56430770e-02, -3.40188369e-02,  3.68277319e-02,
         7.70786032e-02, -5.30065261e-02,  4.77712089e-03,  3.70628908e-02,
         2.19494272e-02,  2.65949755e-03, -2.78133228e-02,  4.56800498e-02,
        -6.83012977e-02, -5.53951897e-02,  1.25349052e-02,  1.50524303e-02,
         7.73917735e-02, -1.00018512e-02,  4.73577380e-02,  7.45450631e-02,
         7.29841879e-04,  4.21403721e-02, -3.87443928e-03,  1.14909917e-01,
        -8.28416422e-02, -7.46538490e-02, -5.30142672e-02, -1.15530670e-01,
        -9.75029245e-02, -2.46479874e-03,  1.87393054e-02, -2.57697348e-02,
        -5.63675212e-03, -9.688

At this time, this service would return a json which one of its entries is the list of chunks created

## Testing Embedding Service on CloudRun

In [35]:
payload={
    "text": text,
    "metadata":{
        "update_date": "Hoy"
    }
}

response = requests.post("https://embedding-service-214571216460.northamerica-south1.run.app/embed-text", json=payload)

In [ ]:
response.text

'{"chunks":[{"vector_id":"577fc1fb-4173-4cdb-b198-721d0b9dc5e4","vector":[0.025923576205968857,0.02159346267580986,-0.05432750657200813,-0.010251697152853012,8.657913713250309e-05,0.0019649912137538195,-0.005383133422583342,-0.027970004826784134,-0.056894250214099884,0.03467877581715584,0.020713230594992638,0.020456204190850258,-0.04093526676297188,-0.04517247900366783,0.0872722938656807,-0.005990470293909311,-0.020912092179059982,0.015643034130334854,-0.03401884436607361,0.036827750504016876,0.07707861065864563,-0.05300649628043175,0.004777187015861273,0.037062861025333405,0.02194947749376297,0.0026595089584589005,-0.02781331166625023,0.045680075883865356,-0.06830135732889175,-0.05539514869451523,0.012534935027360916,0.01505239587277174,0.07739172875881195,-0.010001881048083305,0.04735776409506798,0.07454510033130646,0.0007298100390471518,0.042140357196331024,-0.003874483983963728,0.11490989476442337,-0.08284161239862442,-0.07465384155511856,-0.05301422253251076,-0.11553067713975906,-